[TensorFlow中的变量命名以及命名空间](https://www.cnblogs.com/salan668/p/6994928.html)  
在Tensorflow中, 为了区别不同的变量(例如TensorBoard显示中), 会需要命名空间对不同的变量进行命名. 其中常用的两个函数为: tf.variable_scope, tf.name_scope.

## 定义变量

* tf.variable_scope(name_or_scope,……)  
variable_scope会影响到变量命名。  
A context manager for defining ops that creates variables (layers).
This context manager validates that the (optional) `values` are from the same graph, ensures that graph is the default graph, and pushes a **name scope** and a **variable scope**.
If `name_or_scope` is not None, it is used as is. If `name_or_scope` is None, then `default_name` is used.  In that case, if the same name has been
previously used in the same scope, it will be made unique by **appending `_N`** to it.
Variable scope allows you to create new variables and to share already created ones while providing checks to not create or share by accident. For details,see the @{$variables$Variable Scope How To}, here we present only a few basic examples.

* tf.name_scope(name, default_name=None, values=None)
name_scope不会影响到变量命名。  
A context manager for use when defining a Python op.
This context manager validates that the given `values` are from the
same graph, makes that graph the default graph, and pushes a
name scope in that graph (see @{tf.Graph.name_scope} for more details on that).
* tf.get_variable(name……)
Gets an existing variable with these parameters or create a new one.
This function prefixes the name with the current variable scope
and **performs reuse checks**.如果reuse不为True时，说明变量不允许重用，那么当调用get_variable时，如果同名变量(如果同名变量也是通过get_variable生成的话)已经存在，那么就会报错
>ValueError: Variable foo/bar/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

In [8]:
import tensorflow as tf

#### 变量定义
* tf.Variable定义变量是不会进行reuse check，每次调用都会生成新的变量   
* tf.get_variable会进行reuse check。每次调用前会检查是否用tf.get_variable定义过同名变量，如果没有，就生成新变量；如果已经有同名变量，就进行reuse check，如果为True，直接使用该变量；如果reuse不为True，报错！

In [25]:
x = tf.Variable([1,2], name='X3')
x.name

'X3:0'

In [26]:
x1 = tf.get_variable(name='X3', shape=[1,2])
x1.name

'X3_1:0'

In [18]:
x == x1

False

In [2]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
        assert v.name == "foo/bar/v:0"
print(v.name)

foo/bar/v:0


In [29]:
def foo():
    with tf.variable_scope("foo", reuse=tf.AUTO_REUSE):
        v = tf.get_variable("v", [1])
    return v
v1 = foo()  # Creates v.
v2 = foo()  # Gets the same, existing v.
v1 == v2

True

In [31]:
with tf.variable_scope("foo"):
    a = tf.get_variable("bar1", [1])
print(a.name)

foo/bar1:0


In [3]:
with tf.variable_scope("bar"):
    b = tf.get_variable("bar", [1])
print(b.name)

bar/bar:0


In [33]:
with tf.name_scope("a"):
    a = tf.get_variable("bar1", [1])
    print(a.name)

bar1:0
